### Importing the Necessary Libraries

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow
from tensorflow.data import AUTOTUNE
from tensorflow import keras
from keras.utils import img_to_array, load_img, image_dataset_from_directory
from keras.layers import Rescaling

### Splitting the Dataset into Training Set and Test Set

In [25]:
trainDS = image_dataset_from_directory(
    directory = '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'binary',
    color_mode = 'rgb',
    image_size = (256, 256),
    batch_size = 128,
    shuffle = True,
    seed = 123,
    subset = 'training',
    validation_split = 0.2
)

testDS = image_dataset_from_directory(
    directory = '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'binary',
    color_mode = 'rgb',
    image_size = (256, 256),
    batch_size = 128,
    shuffle = True,
    seed = 123,
    subset = 'validation',
    validation_split = 0.2
)

print(trainDS.class_names)
trainDS = trainDS.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
testDS = testDS.cache().prefetch(buffer_size = AUTOTUNE)

Found 2481 files belonging to 2 classes.
Using 1985 files for training.
Found 2481 files belonging to 2 classes.
Using 496 files for validation.
['COVID', 'non-COVID']


### Rescaling the Training and Test Dataset

In [27]:
rescalingLayer = Rescaling(scale = 1./255)
trainDS = trainDS.map(lambda x, y: (rescalingLayer(x), y))
testDS = testDS.map(lambda x, y: (rescalingLayer(x), y))

tf.Tensor(
[[[[0.00319877 0.00319877 0.00319877]
   [0.00319877 0.00319877 0.00319877]
   [0.00319877 0.00319877 0.00319877]
   ...
   [0.00156292 0.00156292 0.00156292]
   [0.00129142 0.00129142 0.00129142]
   [0.00119954 0.00119954 0.00119954]]

  [[0.00319123 0.00319123 0.00319123]
   [0.00319123 0.00319123 0.00319123]
   [0.00319123 0.00319123 0.00319123]
   ...
   [0.00196873 0.00196873 0.00196873]
   [0.00161652 0.00161652 0.00161652]
   [0.00148603 0.00148603 0.00148603]]

  [[0.00318339 0.00318339 0.00318339]
   [0.00318339 0.00318339 0.00318339]
   [0.00318339 0.00318339 0.00318339]
   ...
   [0.00249112 0.00249112 0.00249112]
   [0.00207946 0.00207946 0.00207946]
   [0.00190882 0.00190882 0.00190882]]

  ...

  [[0.00346715 0.00346715 0.00346715]
   [0.00342185 0.00342185 0.00342185]
   [0.00336932 0.00336932 0.00336932]
   ...
   [0.00333281 0.00333281 0.00333281]
   [0.00337835 0.00337835 0.00337835]
   [0.00337638 0.00337638 0.00337638]]

  [[0.00345237 0.00345237 0.003452